In [25]:
import pandas as pd
import numpy as np

df = pd.read_excel("../데이터자료/join_data.xlsx", index_col=0)

df = df.set_index("DateTime")

df["1Y_베이시스대비"] =df["1Y_베이시스"]-df["1Y_베이시스"].shift(1)
df["2Y_베이시스대비"] =df["2Y_베이시스"]-df["2Y_베이시스"].shift(1)
df["3Y_베이시스대비"] =df["3Y_베이시스"]-df["3Y_베이시스"].shift(1)
df["5Y_베이시스대비"] =df["5Y_베이시스"]-df["5Y_베이시스"].shift(1)
df["10Y_베이시스대비"] =df["10Y_베이시스"]-df["10Y_베이시스"].shift(1)
df["국고1년대비"] =df["국고1년"]-df["국고1년"].shift(1)
df["국고3년대비"] =df["국고3년"]-df["국고3년"].shift(1)
df["국고5년대비"] =df["국고5년"]-df["국고5년"].shift(1)
df["국고10년대비"] =df["국고10년"]-df["국고10년"].shift(1)
df["통안364일대비"] = df["통안364일"] - df["통안364일"].shift(1)
df["통안2년대비"] = df["통안2년"] - df["통안2년"].shift(1)
df["전날 종가_ex"] = df["종가_ex"].shift(1)
df["종가_NDF차이"] = df["전날 종가_ex"] -df["Mid_ndf"]
df = df.dropna()

df2 = df[[ '1Y_베이시스대비', '2Y_베이시스대비', '3Y_베이시스대비', '5Y_베이시스대비', '10Y_베이시스대비'
            ,'M1_스왑포인트',  '국고1년대비', '국고3년대비', '국고5년대비',
             '국고10년대비', '통안364일대비',  '종가_NDF차이',"전날 종가_ex", "종가_ex"]]

df2.head()

,1Y_베이시스대비,2Y_베이시스대비,3Y_베이시스대비,5Y_베이시스대비,10Y_베이시스대비,M1_스왑포인트,국고1년대비,국고3년대비,국고5년대비,국고10년대비,통안364일대비,종가_NDF차이,전날 종가_ex,종가_ex
DateTime,,,,,,,,,,,,,,
2012-08-02,2.0,8.0,9.0,9.0,9.0,250,-0.03,-0.04,-0.04,-0.04,-0.03,-7.50,1126.5,1131.7
2012-08-03,2.0,1.5,1.0,-5.0,-13.0,240,-0.02,-0.02,-0.07,-0.07,-0.02,-6.30,1131.7,1134.8
2012-08-06,-2.0,-4.5,-5.0,-6.0,-5.0,240,0.01,0.01,0.00,0.00,0.01,6.30,1134.8,1129.0
2012-08-07,1.0,1.5,0.0,-8.0,-10.0,250,-0.01,0.00,0.00,0.00,-0.01,0.00,1129.0,1128.8
2012-08-08,0.0,-2.0,-2.0,-4.0,-7.0,240,-0.01,-0.02,-0.02,-0.02,-0.01,-1.45,1128.8,1128.3


In [26]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline 
from matplotlib.pylab import rcParams 
rcParams['figure.figsize']=20,10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dropout,Dense 
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf

## Standard Scaling

In [29]:
x = df[['1Y_베이시스대비', '국고3년대비', '국고5년대비','종가_NDF차이', '전날 종가_ex']]
y = df[["종가_ex"]]

# 이건 이렇게 해야 밑에 코드 8번째 줄 columns에 들어갈 수 있다고 하네요!
x.feature = x.columns 
from sklearn.preprocessing import StandardScaler 



C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_38364\1347824689.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  x.feature = x.columns


,1Y_베이시스대비,국고3년대비,국고5년대비,종가_NDF차이,전날 종가_ex
0,0.348741,-0.646622,-1.079749,-0.495597,-0.149841
1,0.348741,-0.323869,-1.890219,-0.415698,-0.056232
2,-0.350946,0.160261,0.000879,0.423243,-0.000426
3,0.173819,-0.001116,0.000879,0.003773,-0.104837
4,-0.001103,-0.323869,-0.539435,-0.092772,-0.108437
...,...,...,...,...,...
2454,-0.700790,-0.969375,-1.890219,0.213508,3.207485
2455,0.348741,-0.485246,-0.539435,0.183546,3.220086
2456,0.348741,-0.485246,-1.349905,-0.189317,3.110275
2457,0.173819,0.644391,0.811350,0.489826,3.212885


In [28]:
train = x[:-500]
test = x[-500:]

In [34]:
# 새로운 데이터 셋 형성 (훈련, 검증 데이터로 분활) : 2218번째 데이터가 2018년 기말 데이터임
df_last=x.values 
train_df=df_last[:-500] 
test_df=df_last[-500:] 

scaler=MinMaxScaler(feature_range=(0,1)) 
scaled_df=scaler.fit_transform(df_last) 

x_train = [] 
y_train = []
for i in range(60,len(train_df)):
#     # x_train은 2010.1.4부터 60일 후 데이터를 한 묶음으로 시작하여 마지막 데이터 그룹의 끝이 2018년 기말 데이터가 들어가도록 설정 (총 2158개 그룹이 들어감)
    x_train.append(scaled_df[i-60:i,0]) 
     # y_train은 2010.1.4기준 60일 후 데이터로 시작하여 마지막에 2018년 기말 데이터가 들어가도록 설정
    y_train.append(scaled_df[i,0]) 

# # list로 들어간 데이터를 array 형태로 바꿔줌
x_train, y_train = np.array(x_train), np.array(y_train) 
# # reshape를 통해 x_train 데이터를 2158그룹 * 그룹당 60일치 데이터 * 1차원으로 3차원 데이터로 확장함
x_train=np.reshape(x_train,(x_train.shape[0], x_train.shape[1], 1))

print(x_train.shape, y_train.shape)

(1899, 60, 1) (1899,)


In [42]:
x_train

array([[[0.50753769],
        [0.50753769],
        [0.48743719],
        ...,
        [0.48743719],
        [0.50251256],
        [0.50251256]],

       [[0.50753769],
        [0.48743719],
        [0.50251256],
        ...,
        [0.50251256],
        [0.50251256],
        [0.50251256]],

       [[0.48743719],
        [0.50251256],
        [0.49748744],
        ...,
        [0.50251256],
        [0.50251256],
        [0.49246231]],

       ...,

       [[0.44974874],
        [0.3718593 ],
        [0.54773869],
        ...,
        [0.50251256],
        [0.49748744],
        [0.5       ]],

       [[0.3718593 ],
        [0.54773869],
        [0.49748744],
        ...,
        [0.49748744],
        [0.5       ],
        [0.49246231]],

       [[0.54773869],
        [0.49748744],
        [0.51758794],
        ...,
        [0.5       ],
        [0.49246231],
        [0.50251256]]])

In [35]:
# lstm 모델 형성
lstm_model=Sequential() 
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1))) 
lstm_model.add(LSTM(units=50)) 
lstm_model.add(Dense(1))
lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 60, 50)            10400     
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________


In [36]:
# LSTM 시행 (loss는 mse, optimizer는 adam을 사용)
lstm_model.compile(loss='mean_squared_error',optimizer='adam') 
lstm_model.fit(x_train, y_train, epochs=1, batch_size=1)

1899/1899 [==============================] - 94s 45ms/step - loss: 0.0017


In [50]:
# 입력데이터 지정 (2018년 말 - 60일부터 2019년 말까지 값)
input_df = x[len(x) - len(test_df) - 60 : ].values


# 스케일러 적용
input_df = scaler.transform(input_df)

print(input_df.shape)

x_test = [] 
for i in range(60,input_df.shape[0]): 
    x_test.append(input_df[i-60:i,0])


# x_test를 통해 주가 예측 시행 후 scaler 역적용으로 실제 주가 출력
pred = lstm_model.predict(x_test) 
pred = scaler.inverse_transform(pred)

(560, 5)


ValueError: in user code:

    File "c:\Users\tlsdm\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\tlsdm\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\tlsdm\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\tlsdm\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1791, in predict_step
        return self(x, training=False)
    File "c:\Users\tlsdm\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users

In [44]:
input_df

array([[ 0.0000e+00, -2.0000e-02, -2.0000e-02, -3.3000e+00,  1.2297e+03],
       [ 4.0000e+00,  1.0000e-02,  1.0000e-02,  6.1000e+00,  1.2355e+03],
       [ 2.0000e+00,  1.0000e-02,  0.0000e+00,  4.4500e+00,  1.2262e+03],
       ...,
       [ 2.0000e+00, -3.0000e-02, -5.0000e-02, -2.9000e+00,  1.3076e+03],
       [ 1.0000e+00,  4.0000e-02,  3.0000e-02,  7.3000e+00,  1.3133e+03],
       [-4.0000e+00, -1.3000e-01, -1.2000e-01,  3.5000e-01,  1.2961e+03]])

## 시계열 데이터의 데이터셋 분리

## LSTM

In [4]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, LSTM
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
# from tensorflow.keras.datasets import mnist 
# from tensorflow.keras.utils import to_categorical
# from sklearn.model_selection import train_test_split

# import matplotlib.pyplot as plt
# import numpy as np 
# import tensorflow as tf

# tf.random.set_seed(3)

# x_train, x_test, y_train, y_test = train_test_split(x_num, y_num, test_size=0.2, random_state=10)

In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(16, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               return_sequences=False)
          )
model.add(Dense(1))

In [24]:
model.compile(loss='mean_squared_error', optimizer='adam')
# early_stop = EarlyStopping(monitor='val_loss', patience=5)
# filename = path.join(model_path, 'tmp_checkpoint.h5')
# checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit(x_train, y_train, 
                    epochs=200, 
                    batch_size=16,
                    validation_data=(x_valid, y_valid), 
                    callbacks=[early_stop, checkpoint])

Epoch 1/200
95/95 [==============================] - ETA: 0s - loss: 0.1114

AttributeError: 'Sequential' object has no attribute 'format'

In [ ]:
df_last=x_scaled.values 
train_df=df_last[0:2000,:] 
valid_df=df_last[2000:,:] 


x_train = [] 
y_train = []
for i in range(60,len(train_df)):
    # x_train은 2010.1.4부터 60일 후 데이터를 한 묶음으로 시작하여 마지막 데이터 그룹의 끝이 2018년 기말 데이터가 들어가도록 설정 (총 2158개 그룹이 들어감)
    x_train.append(x_scaled[i-60:i,0]) 
    # y_train은 2010.1.4기준 60일 후 데이터로 시작하여 마지막에 2018년 기말 데이터가 들어가도록 설정
    y_train.append(x_scaled[i,0]) 

In [62]:
x_train_3 = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test_3 = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
# y_train_3 = y_train.reshape((y_train.shape[0], y_train.shape[1], 1))
# y_test_3 = y_test.reshape((y_test.shape[0], y_test.shape[1], 1))

In [63]:
x_train_3.shape, x_test_3.shape, y_train.shape, y_test.shape

((1967, 5, 1), (492, 5, 1), (1967, 1), (492, 1))

In [65]:
model = Sequential()
model.add(LSTM(50, activation = "relu", input_shape =(5,1)))
model.add(Dense(1))
model.compile(loss='mse', optimizer = "adam", metrics = ['mae'])

In [66]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 50)                10400     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


In [67]:
history = model.fit(x_train_3, y_train, epochs=30)

Epoch 1/30
62/62 [==============================] - 1s 4ms/step - loss: 1288332.3750 - mae: 1133.6685
Epoch 2/30
62/62 [==============================] - 0s 3ms/step - loss: 950635.5000 - mae: 934.0082
Epoch 3/30
62/62 [==============================] - 0s 4ms/step - loss: 291770.6562 - mae: 167.3161
Epoch 4/30
62/62 [==============================] - 0s 3ms/step - loss: 87912.1719 - mae: 91.7115
Epoch 5/30
62/62 [==============================] - 0s 3ms/step - loss: 50518.0391 - mae: 70.4756
Epoch 6/30
62/62 [==============================] - 0s 3ms/step - loss: 33308.9023 - mae: 61.1219
Epoch 7/30
62/62 [==============================] - 0s 3ms/step - loss: 23691.5820 - mae: 56.1160
Epoch 8/30
62/62 [==============================] - 0s 3ms/step - loss: 18036.2812 - mae: 54.7417
Epoch 9/30
62/62 [==============================] - 0s 3ms/step - loss: 13888.3418 - mae: 51.2172
Epoch 10/30
62/62 [==============================] - 0s 3ms/step - loss: 11290.1611 - mae: 50.3609
Epoch 11/30

## LSTM2

In [73]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.datasets import mnist 
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

tf.random.set_seed(3)

x_train, x_test, y_train, y_test = train_test_split(x_num, y_num, test_size=0.2, random_state=10)